In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import itertools as it

In [19]:
movie = pd.read_csv('movie_bd_v5.csv') # записываем CSV-файл в объект DataFrame
movie.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1829,tt0208003,30000000,173959438,Big Momma's House,Martin Lawrence|Nia Long|Paul Giamatti|Jascha ...,Raja Gosnell,This FBI agent is going undercover... and he's...,When a street-smart FBI agent is sent to Georg...,98,Crime|Comedy,Twentieth Century Fox Film Corporation|Regency...,5/31/2000,5.7,2000
1490,tt0424345,5000000,26888376,Clerks II,Brian O'Halloran|Jeff Anderson|Jason Mewes|Kev...,Kevin Smith,With No Power Comes No Responsibility,A calamity at Dante and Randall's shops sends ...,97,Comedy,The Weinstein Company|View Askew Productions,5/25/2006,6.9,2006
182,tt2397535,5500000,3000000,Predestination,Ethan Hawke|Sarah Snook|Noah Taylor|Christophe...,Michael Spierig|Peter Spierig,To save the future he must reshape the past.,Predestination chronicles the life of a Tempor...,97,Science Fiction|Thriller,Screen Australia|Screen Queensland|Blacklab En...,8/28/2014,7.2,2014
279,tt1111422,100000000,150166126,The Taking of Pelham 1 2 3,Denzel Washington|John Travolta|Luis GuzmÃ¡n|V...,Tony Scott,I can't get it out of my head. I'm gonna die t...,"Armed men hijack a New York City subway train,...",106,Thriller|Drama|Crime,Columbia Pictures|Escape Artists|Scott Free Pr...,6/11/2009,6.1,2009
537,tt0240462,70000000,176104344,Dr. Dolittle 2,Eddie Murphy|Kristen Wilson|Raven-SymonÃ©|Kyla...,Steve Carr,The doctor is in again.,"Dr. John Dolittle the beloved doctor is back, ...",87,Comedy|Family|Romance|Fantasy,Twentieth Century Fox Film Corporation,6/22/2001,4.9,2001


In [20]:
movie.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [21]:
'''
Смотрим, что имеем
00 imdb_id               идентификатор фильма
01 budget                бюджет фильма                  в 1,6,7,8,10,12,16,18  
02 revenue               доход, который принес фильм    в 6,7,8,9,10,12,13,18,19
03 original_title        оригинальное название          в 24,26,27
04 cast                  актерский состав фильма        в 15,16,17,27
05 director              режиссер фильма                в 13,23
06 tagline               слоган фильма                  -  
07 overview              обзор фильма                   в 25
08 runtime               продолжительность фильма (мин) в 2,3,4,5
09 genres                жанры фильма                   в 11,12,17 
10 production_companies  киностудии                     в 18,19,20,25
11 release_date          дата выхода фильма в прокат    в 21,22,23     месяц, сезон
12 vote_average          средний результат голосования  в 26
13 release_year          год выхода (релиза) фильма     в 9,10,15,19,20
'''
print(movie.info())    # смотрим структуру и типы переменных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB
None


# Предобработка

In [129]:
answers = {} # создадим словарь для ответов

'''
Распаковать вложенные списки вида [['Comedy', 'Animation', 'Family'], ['Action', 'Adventure', 'Crime'],...]
в обычный список. Функция для вопросов № 11, 12, 16, 17
'''
def list_unpack(list_nested):
    return list(it.chain.from_iterable(list_nested))


'''
Определить номер сезона по назаванию месяца
'''
def season_ord(m):
    if m == "Dec" or m == "Jan" or m == "Feb":
        return 1 
    if m == "Mar" or m == "Apr" or m == "May":
        return 2 
    if m == "Jun" or m == "Jul" or m == "Aug":
        return 3 
    if m == "Sep" or m == "Oct" or m == "Nov":
        return 4 

In [23]:
movie.drop('tagline', axis=1, inplace=True) # колонка tagline не будет использоваться, можно удалить
movie.drop_duplicates(keep='last')   # если есть дублирующие строки,то удалить, оставив последнее из дублирующихся наблюдений 
movie.dropna(how = 'all')            # то удалить те строки, в которых все значения являются значениями NaN
movie.dropna(how = 'all', axis=1)    # попрощаемся со столбцами,  в которых все значения являются значениями NaN 
movie['profit'] = movie['revenue']-movie['budget'] # добавить колонку прибыль profit = revenue - budget
movie['release_month'] = pd.to_datetime(movie['release_date'], infer_datetime_format=True).dt.strftime('%b') # имя месяца
movie['season'] = movie.release_month.apply(season_ord)  # додобавить колонку с номером сезона (зима-1, весна-2..)
movie.drop('release_date', axis=1, inplace=True)         # колонка release_date больше не нужна, можно удалить
movie['original_title_len'] = movie['original_title'].str.len()            # посчитаем количество символов в названии фильма
movie['overview_words_len']=movie.overview.map(lambda w: len(w.split(' '))) # посчитаем количество слов в обзоре
movie.drop('overview', axis=1, inplace=True)                  # колонка overview объемная и больше не нужна, можно удалить
if len(movie[~movie.imdb_id.str.match("tt", na=False)]) > 0:  # Проверить, сколько imdb_id  начинаются не с tt
    movie.imdb_id.str.match("tt", na=False) 
if len(movie) - movie['imdb_id'].nunique() == 0:              # проверяем на уникальность значений imdb_id
   print('Идентификаторы фильмов уникальны')
else:
   print('Есть повторяющиеся значения в колонке imdb_id') 
movie.shape

Идентификаторы фильмов уникальны


(1889, 16)

In [184]:
'''
Смотрим, что имеем
00 imdb_id               идентификатор фильма
01 budget                бюджет фильма                      в 1,8,10,12,16,18  
02 revenue               доход, который принес фильм        в 8,9,10,12,13,18,19
03 original_title        оригинальное название              в 24,26,27
04 cast                  актерский состав фильма            в 15,16,17,27
05 director              режиссер фильма                    в 13,23
06 runtime               продолжительность фильма (мин)     в 2,3,4,5
07 genres                жанры фильма                       в 11,12,17 
08 production_companies  киностудии                         в 18,19,20,25
09 vote_average          средний результат голосования      в 26
10 release_year          год выхода (релиза) фильма         в 9,10,15,19,20
11 profit                прибыль                            в 6,7
12 release_month         название месяца                    в 21    
13 season                сезон года                         в 22,23 
14 original_title_len    количество символов original_title в 24
15 overview_words_len    количество слов обзора фильма      в 25
'''
print(movie.info())   # структуру и типы переменных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   runtime               1889 non-null   int64  
 7   genres                1889 non-null   object 
 8   production_companies  1889 non-null   object 
 9   vote_average          1889 non-null   float64
 10  release_year          1889 non-null   int64  
 11  profit                1889 non-null   int64  
 12  release_month         1889 non-null   object 
 13  season                1889 non-null   int64  
 14  original_title_len    1889 non-null   int64  
 15  overview_words_len   

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [0]:
# в словарь вставляем номер вопроса и ответ на него, 
# если ответили верно, можете добавить комментарий со значком "+"

In [130]:
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)' # +

In [25]:
movie[movie['budget'] == movie['budget'].max()][['imdb_id', 'original_title', 'budget']]

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


In [26]:
movie[movie.budget == max(movie.budget)][['imdb_id', 'original_title', 'budget']]

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


In [27]:
movie.nlargest(1, ['budget'])[['imdb_id', 'original_title', 'budget']]

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


# 2. Какой из фильмов самый длительный (в минутах)?

In [131]:
answers['2'] = '2. Gods and Generals (tt0279111)' # +

In [29]:
movie[movie['runtime'] == movie['runtime'].max()][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
1157,tt0279111,Gods and Generals,214


In [30]:
movie[movie.runtime == max(movie.runtime)][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
1157,tt0279111,Gods and Generals,214


In [31]:
movie.nlargest(2, ['runtime'])[['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
1157,tt0279111,Gods and Generals,214
1081,tt0167260,The Lord of the Rings: The Return of the King,201


# 3. Какой из фильмов самый короткий (в минутах)?





In [132]:
answers['3'] = '3. Winnie the Pooh (tt1449283)' # +

In [34]:
movie[movie['runtime'] == movie['runtime'].min()][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


In [35]:
movie[movie.runtime == min(movie.runtime)][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


In [36]:
movie.nsmallest(1, ['runtime'])[['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


# 4. Какова средняя длительность фильмов?


In [133]:
answers['4'] = '2. 110' # +

In [38]:
int(round(movie['runtime'].mean(), 0))

110

In [39]:
int(round(movie.runtime.mean(), 0))

110

In [40]:
int(round(movie.runtime.describe()['mean'], 0))

110

# 5. Каково медианное значение длительности фильмов? 

In [134]:
answers['5'] = '1. 107' # +

In [42]:
int(round(movie.runtime.median(), 0))

107

In [43]:
int(round(np.median(movie.runtime), 0))

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [0]:
# лучше код получения столбца profit вынести в Предобработку что в начале

In [135]:
answers['6'] = '5. Avatar (tt0499549)' # +

In [45]:
movie[movie['profit'] == movie['profit'].max()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
239,tt0499549,Avatar,2544505847


In [46]:
movie[movie.profit == max(movie.profit)][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
239,tt0499549,Avatar,2544505847


In [47]:
movie.nlargest(1, ['profit'])[['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
239,tt0499549,Avatar,2544505847


# 7. Какой фильм самый убыточный? 

In [136]:
answers['7'] = '5. The Lone Ranger (tt1210819)' # +

In [49]:
movie[movie['profit'] == movie['profit'].min()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


In [50]:
movie[movie.profit == min(movie.profit)][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


In [51]:
movie.nsmallest(1, ['profit'])[['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [137]:
answers['8'] = '1. 1478' # +

In [53]:
len(movie[movie.profit > 0])

1478

ВАРИАНТ 2

In [54]:
len(movie[movie.revenue > movie.budget])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [138]:
answers['9'] = '4. The Dark Knight (tt0468569)' # +

In [56]:
movie_temp = movie.query('release_year == 2008')[['imdb_id', 'original_title', 'revenue']]
movie_temp[movie_temp.revenue == max(movie_temp.revenue)][['imdb_id', 'original_title', 'revenue']]

,imdb_id,original_title,revenue
599,tt0468569,The Dark Knight,1001921825


ВАРИАНТ 2

In [57]:
movie_temp = movie[movie.release_year == 2008][['imdb_id', 'original_title', 'revenue']]
movie_temp[movie_temp.revenue == max(movie_temp.revenue)][['imdb_id', 'original_title', 'revenue']]

,imdb_id,original_title,revenue
599,tt0468569,The Dark Knight,1001921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [139]:
answers['10'] = '5. The Lone Ranger (tt1210819)'  # +

In [59]:
movie_temp = movie.query('release_year > 2011 & release_year < 2015')[['imdb_id', 'original_title', 'profit']]
movie_temp[movie_temp.profit == min(movie_temp.profit)][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


ВАРИАНТ 2

In [60]:
movie_temp = movie[movie.release_year.isin(range(2012, 2015))][['imdb_id', 'original_title', 'profit']]
movie_temp[movie_temp.profit == min(movie_temp.profit)]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


# 11. Какого жанра фильмов больше всего?

In [0]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [140]:
answers['11'] = '3. Drama'   # +

In [62]:
list_nested = [i.split('|') for i in movie['genres']]   # создаем список без разделителей '|' между названиями жанров
list_genres = Counter(list_unpack(list_nested))         # функцией list_unpack записываем распакованный список и считаем 
list_genres.most_common(1)                              # выводим самое частое значение в списке

[('Drama', 782)]

ВАРИАНТ 2

In [63]:
pd.Series(movie.genres.str.cat(sep='|').split('|')).value_counts().head(1)

Drama    782
dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [141]:
answers['12'] = '1. Drama'   # +

In [65]:
movie_temp = movie[movie['profit'] > 0]                    # отбираем прибыльные фильмы
list_nested = [i.split('|') for i in movie_temp['genres']]
list_genres = Counter(list_unpack(list_nested))
list_genres.most_common(1)

[('Drama', 560)]

ВАРИАНТ 2

In [66]:
movie_temp = movie.query('profit > 0')[['imdb_id', 'genres', 'profit']]
pd.Series(movie_temp.genres.str.cat(sep='|').split('|')).value_counts().head(1)

Drama    560
dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [142]:
answers['13'] = '5. Peter Jackson'   # +

In [68]:
movie.groupby(['director'])['revenue'].sum().sort_values(ascending=False)[:1]

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

ВАРИАНТ 2

In [69]:
director_revenue = Counter()
for i in range(len(movie)):
    list_temp = movie.iloc[i]['director'].split('|')
    for director in list_temp:
        director_revenue[director] += movie.iloc[i]['revenue']
director_revenue.most_common(1)

[('Peter Jackson', 6490593685)]

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [143]:
answers['14'] = '3. Robert Rodriguez'   # +

In [71]:
movie_temp = movie[movie.genres.str.contains('Action')][['director']]
directors = movie_temp.director.str.cat(sep='|').split('|')
pd.Series(directors).value_counts().head(1)

Robert Rodriguez    9
dtype: int64

ВАРИАНТ 2

In [72]:
movie_temp = movie[movie.genres.map(lambda x: 'Action' in x)][['director']]
directors = movie_temp.director.str.cat(sep='|').split('|')
pd.Series(directors).value_counts().head(1)

Robert Rodriguez    9
dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [144]:
answers['15'] = '3. Chris Hemsworth'   # +

In [74]:
movie_temp = movie[movie.release_year == 2012]
list_actor = set(movie_temp.cast.str.cat(sep='|').split('|'))
pd.Series({x: sum(movie_temp[movie_temp.cast.str.contains(x)].revenue) for x in list_actor}).nlargest(1)

Chris Hemsworth    2027450773
dtype: int64

ВАРИАНТ 2

In [75]:
movie_temp = movie.query('release_year == 2012')[['revenue', 'cast']]
list_actor = Counter()
for i in range(len(movie_temp)):
    list_temp = movie_temp.iloc[i]['cast'].split('|')
    for actor in list_temp:
        list_actor[actor] += movie_temp.iloc[i]['revenue']
list_actor.most_common(1)

[('Chris Hemsworth', 2027450773)]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [145]:
answers['16'] = '3. Matt Damon'   # +

In [77]:
movie_temp = movie[(movie['budget'] > movie['budget'].mean())]['cast'] # отбираем фильмы с бюджетом выше среднего и актеров
list_nested = [i.split('|') for i in movie_temp]
list_actor = Counter(list_unpack(list_nested))   
list_actor.most_common(1)

[('Matt Damon', 18)]

ВАРИАНТ 2

In [79]:
movie_temp = movie[(movie['budget'] > movie['budget'].mean())][['imdb_id', 'cast', 'budget']]
list_actor = set(movie_temp.cast.str.cat(sep='|').split('|'))
pd.Series({x: len(movie_temp[movie_temp.cast.str.contains(x)].imdb_id) for x in list_actor}).nlargest(1)

Matt Damon    18
dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [146]:
answers['17'] = '2. Action'   # +

In [81]:
movie_temp = movie[movie['cast'].str.contains('Nicolas Cage')] # отбираем с фильмами в которых снимался Nicolas Cage 
list_nested = [i.split('|') for i in movie_temp['genres']]
list_genres = Counter(list_unpack(list_nested))
list_genres.most_common(1)

[('Action', 17)]

ВАРИАНТ 2

In [82]:
movie_temp = movie[movie.cast.str.contains('Nicolas Cage')].genres
pd.Series(movie_temp.str.cat(sep='|').split('|')).value_counts()[:1]

Action    17
dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [147]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'   # +

In [84]:
paramount_movies = movie[movie['production_companies'].str.contains('Paramount Pictures')]
paramount_movies.sort_values('profit')[['imdb_id', 'original_title','profit']][:1]

,imdb_id,original_title,profit
925,tt0267626,K-19: The Widowmaker,-64831034


ВАРИАНТ 2

In [85]:
movie_temp = movie[movie.production_companies.str.contains('Paramount Pictures')]
movie_temp[movie_temp.profit == min(movie_temp.profit)][['imdb_id', 'original_title','profit']]

,imdb_id,original_title,profit
925,tt0267626,K-19: The Widowmaker,-64831034


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [148]:
answers['19'] = '5. 2015'   # +

In [87]:
movie_temp = movie.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)[:1]

# 20. Какой самый прибыльный год для студии Warner Bros?

In [149]:
answers['20'] = '1. 2014'

In [89]:
movie_temp = movie[movie.production_companies.str.contains('Warner Bros')][['profit', 'release_year']]
movie_temp.groupby(['release_year'])['profit'].sum().sort_values(ascending=False)[:1]

release_year
2014    2295464519
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [150]:
answers['21'] = '4. Сентябрь'  # +

In [91]:
movie.groupby(['release_month'])['imdb_id'].count().sort_values(ascending=False)[:1]

release_month
Sep    227
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [151]:
answers['22'] = '2. 450'   # +

In [93]:
movie.groupby(['season'])['imdb_id'].count()[3]

450

ВАРИАНТ 2

In [94]:
len(movie[movie['season'] == 3]) 

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [152]:
answers['23'] = '5. Peter Jackson'   # +

In [96]:
movie_temp = movie[movie.season == 1][['imdb_id','director']]
movie_temp.groupby(['director'])['imdb_id'].count().sort_values(ascending=False)[:1]

director
Peter Jackson    7
Name: imdb_id, dtype: int64

ВАРИАНТ 2

In [97]:
movie_temp = movie[movie['season'] == 1][['imdb_id','director', 'season']]
pd.Series(movie_temp.director.str.cat(sep='|').split('|')).value_counts()[:1]

Peter Jackson    7
dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [153]:
answers['24'] = '5. Four By Two Productions'   # +

In [99]:
companies = movie.production_companies.str.cat(sep='|').split('|')
companies = pd.Series(companies)
companies = companies.value_counts(ascending=False)
for company in companies.index:
    company_movies = movie['production_companies'].map(lambda x: True if company in x else False)
    companies[company] = movie['original_title_len'][company_movies].mean()
companies = pd.DataFrame(companies).sort_values(0, ascending=False)
companies[:1]

,0
Four By Two Productions,83


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [154]:
answers['25'] = '3. Midnight Picture Show'   # +

In [101]:
companies = movie.production_companies.str.cat(sep='|').split('|')
companies = pd.Series(companies)
companies = companies.value_counts(ascending=False)
for company in companies.index:
    company_movies = movie['production_companies'].map(lambda x: True if company in x else False)
    companies[company] = movie['overview_words_len'][company_movies].mean()
companies = pd.DataFrame(companies).sort_values(0, ascending=False)
companies[:1]

,0
Midnight Picture Show,175


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [155]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'   # +

In [102]:
persent_limit = movie['vote_average'].quantile(0.99)
movie.loc[movie['vote_average'] > persent_limit][['original_title','vote_average']]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [156]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'   # +

In [104]:
actor_and_actor = Counter()
for i in range(len(movie)):
    list_temp = movie.iloc[i]['cast'].split('|')
    for actor1 in list_temp:
        list_new = list_temp.copy()
        list_new.remove(actor1)
        for actor2 in list_new:
            actor_and_actor[(actor1, actor2)] += 1
actor_and_actor.most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

# Submission

In [157]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [158]:
# и убедиться, что ничего не пропустила)
len(answers)

27